In [1]:
import pandas as pd
import datetime as dt

# set dataframe options
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [2]:
# get the data and explore it
data = pd.read_csv("flo_data_20k.csv")
df = data.copy()
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [4]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00,3.11,4.23,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.00,1.91,2.06,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.00,253.92,301.53,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.00,497.32,832.60,12.99,149.98,286.46,578.44,45220.13


In [6]:
df.shape

(19945, 12)

In [7]:
def data_preparing(dataframe):
    # lets figure out how many orders given by each customer
    dataframe["total_order_value"] = dataframe["order_num_total_ever_offline"] + dataframe[
        "order_num_total_ever_online"]
    dataframe["total_value"] = dataframe["customer_value_total_ever_offline"] + dataframe[
        "customer_value_total_ever_online"]

    # change the columns datatypes to datetime which contains date info
    date_type = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_type] = dataframe[date_type].apply(pd.to_datetime)

    return dataframe

In [8]:
df = data_preparing(df)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [10]:
# lets show the distribution within the order channels
df.groupby("order_channel").agg({"master_id": lambda x: x.nunique(),
                                 "total_order_value": lambda x: x.sum(),
                                 "total_value": lambda x: x.sum()})

,master_id,total_order_value,total_value
order_channel,,,
Android App,9495,52269.00,7819062.76
Desktop,2735,10920.00,1610321.46
Ios App,2833,15351.00,2525999.93
Mobile,4882,21679.00,3028183.16


In [11]:
#lets show the top 10 paying customers
df.sort_values(by="total_value", ascending= False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order_value,total_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00,2.00,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00,45905.10
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.00,1.00,130.49,36687.80,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.00,36818.29
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00,1.00,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00,33918.10
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.00,1.00,538.94,30688.47,"[ERKEK, KADIN, AKTIFSPOR]",11.00,31227.41
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.00,3.00,18119.14,2587.20,[AKTIFSPOR],4.00,20706.34
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00,4.00,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.00,18443.57
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00,1.00,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00,16918.57
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.00,1.00,180.73,12545.37,"[ERKEK, AKTIFSPOR]",37.00,12726.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00,1.00,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00,12282.24
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.00,2.00,64.97,12038.18,"[ERKEK, KADIN]",20.00,12103.15


In [12]:
df.sort_values(by="total_order_value", ascending= False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order_value,total_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00,2.00,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00,45905.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00,1.00,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00,12282.24
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.00,109.00,10239.46,143.98,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.00,10383.44
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.00,87.00,8432.25,139.98,[ERKEK],88.00,8572.23
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.00,81.00,3997.55,242.81,[ERKEK],83.00,4240.36
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00,1.00,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00,33918.10
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.00,74.00,4640.77,543.28,"[KADIN, AKTIFSPOR]",77.00,5184.05
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.00,3.00,292.93,5004.95,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.00,5297.88
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00,1.00,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00,16918.57
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00,4.00,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.00,18443.57


In [13]:
# RFM metrics calculations (Recency, Frequency, Monetary)
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [14]:
today_date = dt.datetime(2021,6,1)

In [15]:
rfm = df.groupby("master_id").agg({"last_order_date": lambda x: (today_date-x.max()).days,
                                  "total_order_value": "sum",
                                  "total_value": "sum"})
rfm.columns = ["recency", "frequency", "monetary"]
rfm.head()

,recency,frequency,monetary
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00,776.07
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00,269.47
000be838-85df-11ea-a90b-000d3a38a36f,213,4.00,722.69
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00,874.16
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00,1620.33


In [16]:
rfm.reset_index(inplace=True)

In [17]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,19945.00,134.46,103.28,2.00,43.00,111.00,202.00,367.00
frequency,19945.00,5.02,4.74,2.00,3.00,4.00,6.00,202.00
monetary,19945.00,751.24,895.40,44.98,339.98,545.27,897.78,45905.10


In [18]:
# Calculate the RFM score
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels = [5,4,3,2,1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels = [1,2,3,4,5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels = [1,2,3,4,5])

In [19]:
rfm["RF_score"] = rfm["recency_score"].astype(str)+ rfm["frequency_score"].astype(str)
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_score
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00,776.07,5,4,4,54
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00,269.47,1,2,1,12
2,000be838-85df-11ea-a90b-000d3a38a36f,213,4.00,722.69,2,3,4,23
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00,874.16,5,4,4,54
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00,1620.33,5,4,5,54


In [20]:
# according to rf score make segmentation
seg_map = {
        r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
          }

In [21]:
rfm["segment"] = rfm['RF_score'].replace(seg_map, regex =True)

In [22]:
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_score,segment
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00,776.07,5,4,4,54,champions
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00,269.47,1,2,1,12,hibernating
2,000be838-85df-11ea-a90b-000d3a38a36f,213,4.00,722.69,2,3,4,23,at_Risk
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00,874.16,5,4,4,54,champions
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00,1620.33,5,4,5,54,champions


In [23]:
# investigate rfm values' means 
rfm.groupby("segment").agg({"recency": ["mean", "count"],
                            "frequency": ["mean", "count"],
                            "monetary": ["mean", "count"]}).T

segment          about_to_sleep  at_Risk  cant_loose  champions  hibernating  \
recency   mean           114.03   242.33      235.16      17.14       247.43   
          count         1643.00  3152.00     1194.00    1920.00      3589.00   
frequency mean             2.41     4.47       10.72       8.97         2.39   
          count         1643.00  3152.00     1194.00    1920.00      3589.00   
monetary  mean           361.65   648.33     1481.65    1410.71       362.58   
          count         1643.00  3152.00     1194.00    1920.00      3589.00   

segment          loyal_customers  need_attention  new_customers  \
recency   mean             82.56          113.04          17.98   
          count          3375.00          806.00         673.00   
frequency mean              8.36            3.74           2.00   
          count          3375.00          806.00         673.00   
monetary  mean           1216.26          553.44         344.05   
          count          3375.00          806.00         673.00   

segment          potential_loyalists  promising  
recency   mean                 36.87      58.69  
          count              2925.00     668.00  
frequency mean                  3.31       2.00  
          count              2925.00     668.00  
monetary  mean                533.74     334.15  
          count              2925.00     668.00

In [24]:
cond = df[["master_id", "interested_in_categories_12"]]
cond.head()

,master_id,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,[AKTIFSPOR]


In [25]:
a_final = pd.DataFrame(cond.merge(rfm, how="inner", on="master_id"))
a_final

,master_id,interested_in_categories_12,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_score,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,[KADIN],95,5.00,939.37,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",105,21.00,2013.55,3,5,5,35,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,"[ERKEK, KADIN]",186,5.00,585.32,2,4,3,24,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,"[AKTIFCOCUK, COCUK]",135,2.00,121.97,3,1,1,31,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,[AKTIFSPOR],86,2.00,209.98,3,1,1,31,about_to_sleep
...,...,...,...,...,...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,"[ERKEK, AKTIFSPOR]",331,3.00,401.96,1,2,2,12,hibernating
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,[AKTIFSPOR],161,2.00,390.47,2,1,2,21,hibernating
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,[AKTIFSPOR],8,3.00,632.94,5,2,3,52,potential_loyalists
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,"[ERKEK, AKTIFSPOR]",108,6.00,1009.77,3,4,4,34,loyal_customers


In [26]:
# loyals and champs who interested in woman category
a_final = a_final[a_final["segment"].isin(["champions", "loyal_customers"]) & 
                  (a_final["interested_in_categories_12"].str.contains("KADIN"))]
a_final

,master_id,interested_in_categories_12,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_score,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,[KADIN],95,5.00,939.37,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",105,21.00,2013.55,3,5,5,35,loyal_customers
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,"[ERKEK, COCUK, KADIN]",86,5.00,1054.69,3,4,5,34,loyal_customers
11,c2e15af2-9eed-11e9-9897-000d3a38a36f,"[COCUK, KADIN, AKTIFSPOR]",13,20.00,2861.60,5,5,5,55,champions
33,fb840306-1219-11ea-a001-000d3a38a36f,"[ERKEK, KADIN, AKTIFSPOR]",116,8.00,923.64,3,5,4,35,loyal_customers
...,...,...,...,...,...,...,...,...,...,...
19912,f63aba0e-41d8-11ea-96d9-000d3a38a36f,[KADIN],135,8.00,1084.85,3,5,5,35,loyal_customers
19913,81d0da04-a53e-11e9-a2fc-000d3a38a36f,"[KADIN, AKTIFSPOR]",3,7.00,1152.11,5,5,5,55,champions
19914,825e6b00-ab40-11e9-a2fc-000d3a38a36f,"[KADIN, AKTIFSPOR]",43,7.00,969.89,4,5,4,45,loyal_customers
19917,f8c471c8-2596-11eb-81e9-000d3a38a36f,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",7,7.00,1014.94,5,5,4,55,champions


In [27]:
a_final["master_id"].to_csv("champs_and_loyals_interested_in_woman_cat.csv", index=False)

In [28]:
# cant_loose, about_to_sleep and new_customers who interested in man or kids categories
b_final = pd.DataFrame(cond.merge(rfm, how="inner", on="master_id"))
b_final = b_final[b_final["segment"].isin(["cant_loose","about_to_sleep", "new_customers"]) &
((b_final["interested_in_categories_12"].str.contains("ERKEK")) |(b_final["interested_in_categories_12"].str.contains("COCUK")))]

In [29]:
b_final

,master_id,interested_in_categories_12,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_score,segment
3,1854e56c-491f-11eb-806e-000d3a38a36f,"[AKTIFCOCUK, COCUK]",135,2.00,121.97,3,1,1,31,about_to_sleep
15,13ed97a4-b167-11e9-89fa-000d3a38a36f,"[AKTIFCOCUK, COCUK, KADIN]",288,8.00,932.36,1,5,4,15,cant_loose
23,4fbb0cbe-9085-11ea-af25-000d3a38a36f,"[COCUK, KADIN]",123,2.00,242.45,3,1,1,31,about_to_sleep
34,1acf7870-87af-11ea-ace9-000d3a38a36f,"[ERKEK, KADIN]",103,2.00,298.09,3,1,1,31,about_to_sleep
56,92a334ee-58f2-11eb-9e65-000d3a38a36f,"[ERKEK, KADIN]",110,2.00,214.86,3,1,1,31,about_to_sleep
...,...,...,...,...,...,...,...,...,...,...
19900,b23c30c0-5e80-11ea-be5b-000d3a38a36f,"[ERKEK, KADIN]",126,3.00,224.97,3,2,1,32,about_to_sleep
19932,13a5e98c-a824-11e9-a2fc-000d3a38a36f,"[ERKEK, KADIN, AKTIFSPOR]",208,9.00,1174.00,2,5,5,25,cant_loose
19933,6d285c64-5e73-11ea-be5b-000d3a38a36f,"[AKTIFCOCUK, ERKEK, COCUK]",220,8.00,1114.80,2,5,5,25,cant_loose
19935,b514d468-5b52-11ea-85ca-000d3a38a36f,[ERKEK],105,2.00,259.99,3,1,1,31,about_to_sleep


In [30]:
b_final["master_id"].to_csv("cant_loose_sleeper_new_cust_interested_in_man_or_kids_cat.csv", index=False)